In [1]:
#needs to be same between py and arduino

In [2]:
import Crypto
import Crypto.Random
from Crypto.Cipher import AES

def pad_data(data):
    if len(data) % 16 == 0:
        return data
    databytes = bytearray(data)
    padding_required = 15 - (len(databytes) % 16)
    databytes.extend(b'\x80')
    databytes.extend(b'\x00' * padding_required)
    return bytes(databytes)

def unpad_data(data):
    if not data:
        return data

    data = data.rstrip(b'\x00')
    if data[-1] == 128: # b'\x80'[0]:
        return data[:-1]
    else:
        return data

def generate_aes_key():
    rnd = Crypto.Random.OSRNG.posix.new().read(AES.block_size)
    return rnd

def encrypt(key, iv, data):
    aes = AES.new(key, AES.MODE_CBC, iv)
    #data = pad_data(data) //diff padding py<>uno so make it always 16 or error (no padding)
    return aes.encrypt(data)

def decrypt(key, iv, data):
    aes = AES.new(key, AES.MODE_CBC, iv)
    data = aes.decrypt(data)
    return data #unpad_data(data)

In [6]:
def test_crypto ():
    print("Same IVs same key:")
    
    #key = generate_aes_key()
    #print(key)
   
    key = [ 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15 ]
    key = bytes(key)
    #iv = b"1234567890123456"
    iv = [ 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15 ]
    iv = bytes(iv)

    msg = b"1234567891923454"
    code = encrypt(key, iv, msg)
    print(code.hex())
    decoded = decrypt(key, iv, code)
    print(decoded)

    code = encrypt(key, iv, msg)
    print(code.hex())
    decoded = decrypt(key, iv, code)
    print(decoded)
    
    '''print("")
    print("Different IVs same key:")
    iv = generate_aes_key()
    code = encrypt(key, iv, msg)
    print(code.hex())
    decoded = decrypt(key, iv, code)
    print(decoded)

    iv = generate_aes_key()
    code = encrypt(key, iv, msg)
    print(code.hex())
    decoded = decrypt(key, iv, code)
    print(decoded)'''

In [7]:
test_crypto()

Same IVs same key:
d695f98131c232e2e510b7b262950eab
b'1234567891923454'
d695f98131c232e2e510b7b262950eab
b'1234567891923454'


Arduino
#include <AESLib.h>

uint8_t key[] = { 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15 };

uint8_t iv[] = { 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15 };

memcpy(data, "1234567891923454", 16);

gives:
    
15:03:30.490 -> iv: 000102030405060708090A0B0C0D0E0F

15:03:38.250 -> encrypted-cbc: D695F98131C232E2E510B7B262950EAB

15:03:38.250 -> decrypted-cbc: 1234567891923454

In [9]:
'Yes D695F98131C232E2E510B7B262950EAB == d695f98131c232e2e510b7b262950eab'

'Yes D695F98131C232E2E510B7B262950EAB == d695f98131c232e2e510b7b262950eab'